In [6]:
import pandas as pd
import numpy as np
#data=pd.read_csv('train.csv', sep='\t', encoding='utf8', error_bad_lines=False)
#error_bad_lines=False
#unsup = pd.read_csv('other.csv', sep='\t', encoding='utf8', error_bad_lines=False)

In [7]:
from multiprocessing import cpu_count
print( 'число процессоров = {}'.format( cpu_count() ) )

число процессоров = 8


In [8]:
# 1. Очистим данные от HTML
#for i in data1 ['description']:
#    data1['description'] = data1['description'].replace(i, BeautifulSoup(i, 'html5lib').get_text())
#data1.info()
#data1.to_csv('data1.csv', sep='\t', encoding='utf-8', index=False)

In [9]:
# Чтобы не ждать каждый раз загружаем датафрейс уже без HTML тегов

data1=pd.read_csv('data1.csv', sep='\t', encoding='utf8')

In [10]:
%%time
y = data1['description']
from sklearn.model_selection import train_test_split
train, test, y_train, y_test = \
train_test_split(data1, y, test_size=0.33, random_state=1)

Wall time: 1.51 s


In [11]:
%%time
# Функция которая приводит к нижнему регистру и делит текст на слова
from bs4 import BeautifulSoup
import re

def text_to_wordlist(text): 
    text = re.sub('[^а-яА-ЯёЁ]', ' ', text)
    words = text.lower().split()
    return words

Wall time: 0 ns


функция которая для представленного текста:

удаляет html теги
производит деления на предложения
каждое предложение делит на слова

In [12]:
%%time
import nltk.data
def text_to_sentences(text):
    #text = BeautifulSoup(text).get_text()
    cleanr = re.compile('<.*?>')
    text = re.sub(cleanr, ' ', text)
    tokenizer = nltk.data.load('russian.pickle')
    raw_sentences = tokenizer.tokenize(text.strip())
    
    sentences = []
    for raw_sentence in raw_sentences:
        if len(raw_sentence) > 0:
            sentences.append(text_to_wordlist(raw_sentence))
    return sentences

Wall time: 0 ns


In [13]:
# Все вместе:


In [14]:
%%time

sentences = []

for descr in train['description']:
    sentences += text_to_sentences(descr)

Wall time: 56.7 s


In [15]:
#%%time
#for descr in unsup['description']:
#    sentences += text_to_sentences(descr)

In [16]:
%%time
for descr in test['description']:
    sentences += text_to_sentences(descr)

Wall time: 28.1 s


In [17]:

# В конце сбора sentences будет list list'ов (список списков) - как и пример выше.
# (Повторюсь) каждый элемент списка sentences - предложение, но представленное в виде списка слов - потому список

# выведем количество элементов этого массива (оно же - количество предложений во всех текста)
#print len(sentences)

In [18]:
###############Обучим же теперь модель Word2Vec

In [19]:
%%time
# симортируем соответствующую функцию из модуля gensim, который должен быть установлен

from gensim.models.word2vec import Word2Vec 
from gensim import models
from gensim.models import KeyedVectors

# список параметров, которые можно менять по вашему желанию
num_features = 150  # итоговая размерность вектора каждого слова
min_word_count = 5  # минимальная частотность слова, чтобы оно попало в модель
num_workers = 8     # количество ядер вашего процессора, чтоб запустить обучение в несколько потоков
context = 10        # размер окна 
downsampling = 1e-5 # внутренняя метрика модели

model = Word2Vec(sentences, workers=num_workers, size=num_features,
                 min_count=min_word_count, window=context, sample=downsampling)

C:\Users\Ildar\Anaconda3\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


Wall time: 1min 10s


In [20]:
# Финализируем нашу модель. Ее нельзя будет доучить теперь, но она начнет занимать гораздо меньше места
model.init_sims(replace=True)

##########
#Натренировав модель, получили представление каждого слова в семантическом пространстве (часто называют "псевдо" семантическое пространство)

Т.к. мы хотим классифицировать не слова, а тексты, надо перевести тексты в вектора (представить в виде фич) Один из простых методов - сложить все вектора слов входящих в текст и поделить на число входящих слов. Напишем функцию, которая:

создает нулевой вектор - это будет результирующий вектор
идем по всем словам в тексте, если слово есть в моделе:
увеличиваем счетчик слов
прибавим вектор слова к результирующему вектору
поделим все координаты на число слов по которым, вектора которых мы прибавляли к результирующему вектору

In [21]:
%%time
import numpy as np
def text_to_vec(words, model, size):
    text_vec = np.zeros((size,), dtype="float32")
    n_words = 0

    index2word_set = set(model.wv.index2word)
    for word in words:
        if word in index2word_set:
            n_words = n_words + 1
            text_vec = np.add(text_vec, model[word])
    
    if n_words != 0:
        text_vec /= n_words
    return text_vec

Wall time: 0 ns


Напишем функцию, которая на входе получает список всех текстов, а на выходе отдает список вектор каждого текста - что является прямоугольной матрицей

In [22]:
%%time
def texts_to_vecs(texts, model, size):
    texts_vecs = np.zeros((len(texts), size), dtype="float32")
    
    for i, text in enumerate(texts):
        texts_vecs[i] = text_to_vec(text, model, size)

    return texts_vecs

Wall time: 0 ns


Заметим что функция texts_to_vecs принимает не просто тексты, а список всех слов текста.

(!!!) Внимание: не список списков (там где сначала делили на предложения, а предложения на слова), а обычный линейный список

Но у нас есть функции, которые переводят 1) текст в список предложений, 2) предложение в список слов

Может показаться, что можно использовать 2ую функцию, но придется тогда ее переписать, потому как теги у нас удаляются лишь в первой функции

In [23]:
%%time
# Поступим иначе, в python есть возможность развернуть двухмерный массив в одномерный, вот пример
temp_list = [[1, 2, 3], [4, 5], [6, 7, 8, 9]]
print (sum(temp_list, []))

# магия :)


[1, 2, 3, 4, 5, 6, 7, 8, 9]
Wall time: 823 µs


In [24]:
%%time
# действительно работает, сделаем для всех текстов из train
train_like_word_list = [sum(text_to_sentences(text), []) for text in train['description']]

Wall time: 1min


In [25]:
%%time
train_vecs = texts_to_vecs(train_like_word_list, model, num_features)

C:\Users\Ildar\Anaconda3\lib\site-packages\ipykernel_launcher.py:10: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  # Remove the CWD from sys.path while we load stuff.


Wall time: 22min 52s


In [ ]:
%%time
# сделаем тоже самое для test
test_like_word_list = [sum(text_to_sentences(text), []) for text in test['description']]

In [ ]:
%%time
test_vecs = texts_to_vecs(test_like_word_list,model, num_features)

In [ ]:
# Воспользуемся train_vecs, test_vecs, train['target'] 
#    как матрица фичей обучающей выборки, матрица фичей тестовой выборки, таргет для обучающей выборки соответственно

# Стандартный случайный лес в таком случае
from sklearn.ensemble import RandomForestClassifier

forest = RandomForestClassifier(n_estimators=100, n_jobs=8)
forest = forest.fit(train_vecs, train['target'])
predict = forest.predict(test_vecs)

# И вот задача решена

##########################################################
##########################################################

Но что если для получения результирующего вектора не складываеть вектора, а пойти другим способом.

Кластеризуем все слова на 1000 классов.

In [ ]:
# В model.syn0 хранятся все вектора. Кластеризируем их!

print ('Размер ', model.wv.syn0.shape)
print ('Вектор векторов ', model.wv.syn0)

In [ ]:
w=model.wv.syn0

In [26]:
%%time
# Кластеризируем все слова. 
from gensim.models import KeyedVectors
from sklearn.cluster import KMeans

word_vectors = model.wv.syn0
# Число кластеров установим в 10000. Для этого числа нет "серебряной пули". Для каждого случая лучше подойдет разная
num_clusters = 1000

# Начнем кластеризацию, учитывая что классов много, количество векторов (по сути слов) много,
#   все это будет происходит продолжительное время. Можно сходить за чаем.
kmeans_clustering = KMeans(n_clusters=num_clusters)
idx = kmeans_clustering.fit_predict(word_vectors)

# в idx будут храниться номера классов для каждого слова

C:\Users\Ildar\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: DeprecationWarning: Call to deprecated `syn0` (Attribute will be removed in 4.0.0, use self.wv.vectors instead).
  """
C:\Users\Ildar\Anaconda3\lib\site-packages\sklearn\metrics\pairwise.py:257: RuntimeWarning: invalid value encountered in sqrt
  return distances if squared else np.sqrt(distances, out=distances)


Wall time: 15min 36s


In [27]:
%%time
# создадим структуру dict (словарь): слово -> класс
word_centroid_map = dict(zip(model.wv.index2word, idx))

Wall time: 25.7 ms


In [28]:
%%time
# выведем представителей первых 10 классов и посмотрим на адекватность произошедшей кластеризации
xrange=range
for cluster in xrange(0,10):
    print (cluster)
    #words = []
    #for i in xrange(0, len(word_centroid_map.values())):
    #    if word_centroid_map.values()[i] == cluster:
    #        words.append(word_centroid_map.keys()[i])
    #print (words)
    
    
    words = []
for key, item in word_centroid_map.items():
    if item == cluster:
        words.append(key)
print (words)


#или
#words = [k for k, w in word_centroid_map.items() if w == cluster]
    
    
    #########################

0
1
2
3
4
5
6
7
8
9
['найдет', 'выбирает', 'свете', 'сделав', 'вкусную', 'заставляют', 'живи', 'домашними', 'душа', 'демонстрирует', 'бриллиантов', 'оригинальным', 'расставаться', 'приветливая', 'позволяя', 'индивидуальности', 'особый', 'подарка', 'комфортом', 'ребенку', 'вещь', 'вернуть', 'мозга', 'способна', 'предлагаемый', 'маленькими', 'биться', 'демократичной', 'наполненными', 'окружающий', 'легкого', 'прожить', 'однажды', 'остался', 'здоровому', 'подчеркнуть', 'прибыльным', 'удовлетворяя', 'воплощает', 'меняя', 'радушно', 'слышит', 'мужчины', 'искренней', 'оставаясь', 'сориентироваться', 'слушает', 'незаменимым', 'покупками', 'удобству', 'убрать', 'творцом', 'близкими', 'долгую', 'ощущения', 'вдохновляющий', 'подготовленный', 'создай', 'девушкам', 'лояльно', 'требовательного', 'торговать', 'восторг', 'освещать', 'насладиться', 'женщине', 'посоветовать', 'эстетичным', 'наполненный', 'найдутся', 'подарить', 'самими', 'шедевр', 'придать', 'неизменным', 'превратиться', 'здоровом', 'э


Существует несколько подходов к работе с кластерами слов. Рассмотрим 2 примера

1) посчитаем для каждого текста вектор сколько его слов встретилось в каждом кластере. Т.е. для каждого текста будет вектор размера равного числу кластеров, значениями вектора будут натуральные числа
2) Посчитаем усредненных удаленность векторов текстов от каждого центройда всех кластеров.

In [29]:
# для первого случая напишем функцию, на вход которой поступает текст, представленный в виде списка всех его слов
#    смотрим в каком кластере находится каждое слово 
#    и увеличиваем соответствующую ячейку (ответственную за этот кластер) на 1


def create_bag_of_centroids(wordlist, word_centroid_map, num_centroids):
    bag_of_centroids = np.zeros(num_centroids, dtype="float32")
    set_word_centroid_map = set(word_centroid_map.keys())
    
    for word in wordlist:
        if word in set_word_centroid_map:
            index = word_centroid_map[word]
            bag_of_centroids[index] += 1
    
    return bag_of_centroids

In [30]:
%%time
# Но нам нужно это не для одного текста, а для всех текстов обучающей и тестовой выборки
# Сделаем это

train_vecs_centroids = np.zeros((train['description'].size, num_clusters), dtype="float32")

for i, text in enumerate(train_like_word_list):
    train_vecs_centroids[i] = create_bag_of_centroids(text, word_centroid_map, num_clusters)

#test_vecs_centroids = np.zeros((test['description'].size, num_clusters), dtype="float32")
#
#for i, text in enumerate(test_like_word_list):
#    test_vecs_centroids[i] = create_bag_of_centroids(text, word_centroid_map, num_clusters)
#    
# Результатом будет матрицы для обучающих и тестовых текстов, что будет матрицей фич обучающей и тестовой выборок соотв.

Wall time: 21min 43s


In [31]:
# стандартный случайный лес на полученных матрицах

forest = RandomForestClassifier(n_estimators=100, n_jobs=8)
forest = forest.fit(train_vecs_centroids, train['target'])
predict = forest.predict(test_vecs_centroids)

In [45]:
# а теперь попробуем посчитать растояние от центройдов

train_vecs_centroids_dist = np.zeros((train['description'].size, num_clusters), dtype="float32")

from scipy.spatial import distance

for i, vec in enumerate(train_vecs):
    for j, center in enumerate(kmeans_clustering.cluster_centers_):
        train_vecs_centroids_dist[i][j] = distance.euclidean(vec, center)

In [ ]:
train_vecs_centroids_dist.shape

In [ ]:
# Всё что не нужно для итогового предсказания - закомментил. Чтобы не увеличивать время работы

In [ ]:
#test_vecs_centroids_dist = np.zeros((test['description'].size, num_clusters), dtype="float32")
#
#for i, vec in enumerate(test_vecs):
#    for j, center in enumerate(kmeans_clustering.cluster_centers_):
#        test_vecs_centroids_dist[i][j] = distance.euclidean(vec, center)

In [ ]:
#test_vecs_centroids_dist.shape

In [36]:

# стандартный случайный лес на полученных матрицах (кто бы сомневался)

#forest = RandomForestClassifier(n_estimators=100, n_jobs=8)
#forest = forest.fit(train_vecs_centroids_dist, train['target'])
#predict = forest.predict(test_vecs_centroids_dist)

In [37]:
# МЕТОД ЛОГИЧЕСКОЙ РЕГРЕССИИ

#from sklearn.linear_model import LinearRegression, LogisticRegression
#logreg = LogisticRegression(C=100, class_weight=None, dual=False, fit_intercept=True,
#          intercept_scaling=50, max_iter=10, multi_class='ovr', n_jobs=1,
#          penalty='l2', random_state=None, solver='liblinear', tol=1e-6,
#          verbose=0, warm_start=False)
##обучение логической регрессии
#logreg = logreg.fit(train_vecs_centroids, train['target'])
#predict = logreg.predict(test_vecs_centroids)
#predict2 = logreg.predict(test_vecs_centroids_dist)

In [38]:
# После того как мы обучили модель сделаем предсказания для тестовой выборки


In [32]:
#datatest=pd.read_csv('test.csv', sep='\t', encoding='utf8')

#for i in datatest ['description']:
#    datatest['description'] = datatest['description'].replace(i, BeautifulSoup(i, 'html5lib').get_text())
#datatest.info()
#datatest.to_csv('datatest1.csv', sep='\t', encoding='utf-8', index=False)

# Чтобы не считать каждый роз - загружаю уже готовую выборку без HTML
datatest=pd.read_csv('datatest1.csv', sep='\t', encoding='utf8')


In [33]:
%%time
for descr in datatest['description']:
    text_to_sentences(descr)

Wall time: 1min 5s


In [34]:
%%time
# действительно работает, сделаем для всех текстов из train
test_like_word_list = [sum(text_to_sentences(text), []) for text in datatest['description']]

Wall time: 1min 11s


In [35]:
%%time
test_vecs = texts_to_vecs(test_like_word_list, model, num_features)

C:\Users\Ildar\Anaconda3\lib\site-packages\ipykernel_launcher.py:10: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  # Remove the CWD from sys.path while we load stuff.


Wall time: 29min 32s


In [36]:
%%time

from scipy.spatial import distance
test_vecs_centroids = np.zeros((datatest['description'].size, num_clusters), dtype="float32")

for i, text in enumerate(test_like_word_list):
    test_vecs_centroids[i] = create_bag_of_centroids(text, word_centroid_map, num_clusters)
    
# Результатом будет матрица для  текста, что будет матрицей фич

Wall time: 27min 48s


In [37]:
test_vecs_centroids_dist = np.zeros((datatest['description'].size, num_clusters), dtype="float32")

for i, vec in enumerate(test_vecs):
    for j, center in enumerate(kmeans_clustering.cluster_centers_):
        test_vecs_centroids_dist[i][j] = distance.euclidean(vec, center)

In [38]:
test_vecs_centroids_dist.shape

(170179, 1000)

In [40]:
# Стандарт случ лес
from sklearn.ensemble import RandomForestClassifier
forest = RandomForestClassifier(n_estimators=100, n_jobs=8)
forest = forest.fit(train_vecs, train['target'])
predict = forest.predict(test_vecs)

In [41]:
# стандартный случайный лес на полученных матрицах

forest = RandomForestClassifier(n_estimators=100, n_jobs=8)
forest = forest.fit(train_vecs_centroids, train['target'])
predict1 = forest.predict(test_vecs_centroids)

In [46]:
# стандартный случайный лес на полученных матрицах (кто бы сомневался)

forest = RandomForestClassifier(n_estimators=100, n_jobs=8)
forest = forest.fit(train_vecs_centroids_dist, train['target'])
predict2 = forest.predict(test_vecs_centroids_dist)

In [50]:
# Запишем все по файлам для загрузки на Kaggle
df_id = datatest['id']

df_predict = pd.DataFrame(predict)
df_pr=df_predict[0]
# Объединяем таргет и ID в датафрейм
df_predict1=pd.concat([df_id, df_pr], axis=1)
df_predict1.columns = ['id', 'target']
df_predict1.to_csv('predict1.csv', sep=',', encoding='utf-8', index=False)

df_predict = pd.DataFrame(predict1)
df_pr=df_predict[0]
# Объединяем таргет и ID в датафрейм
df_predict2=pd.concat([df_id, df_pr], axis=1)
df_predict2.columns = ['id', 'target']
df_predict2.to_csv('predict2.csv', sep=',', encoding='utf-8', index=False)

df_predict = pd.DataFrame(predict2)
df_pr=df_predict[0]
# Объединяем таргет и ID в датафрейм
df_predict3=pd.concat([df_id, df_pr], axis=1)
df_predict3.columns = ['id', 'target']
df_predict3.to_csv('predict3.csv', sep=',', encoding='utf-8',index=False)

In [48]:
# МЕТОД ЛОГИЧЕСКОЙ РЕГРЕССИИ

from sklearn.linear_model import LinearRegression, LogisticRegression
logreg = LogisticRegression(C=10, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=50, max_iter=10, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=1e-6,
          verbose=0, warm_start=False)
#обучение логической регрессии
logreg = logreg.fit(train_vecs_centroids, train['target'])
predict2_1 = logreg.predict(test_vecs_centroids)
predict2_2 = logreg.predict(test_vecs_centroids_dist)

In [51]:
df_predict = pd.DataFrame(predict2_1)
df_pr=df_predict[0]
# Объединяем таргет и ID в датафрейм
df_predict=pd.concat([df_id, df_pr], axis=1)
df_predict.columns = ['id', 'target']
df_predict.to_csv('predict2_1.csv', sep=',', encoding='utf-8',index=False)

df_predict = pd.DataFrame(predict2_2)
df_pr=df_predict[0]
# Объединяем таргет и ID в датафрейм
df_predict=pd.concat([df_id, df_pr], axis=1)
df_predict.columns = ['id', 'target']
df_predict.to_csv('predict2_2.csv', sep=',', encoding='utf-8',index=False)